In [1]:
import pandas as pd
import numpy as np
import warnings
warnings.filterwarnings("ignore")
from datetime import datetime
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib notebook
%matplotlib inline
import seaborn as sns
import pickle
from tqdm import tqdm
import math
import os
from sklearn import metrics
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import GridSearchCV
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.multioutput import RegressorChain
from sklearn.model_selection import RandomizedSearchCV
from datetime import datetime
from numpy import mean
from numpy import std
from numpy import absolute
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import RepeatedKFold
from sklearn.multioutput import MultiOutputRegressor
from xgboost import XGBRegressor
from lightgbm import LGBMRegressor
import pickle

In [2]:
file= 'E:\CIT\MSc_AI\Research project_NIMBUS\Rework\corrected_data.xlsx'
#data = pd.read_excel(file,infer_datetime_format=True,index_col=0,sheet_name='data_2019')
data = pd.read_excel(file,sheet_name='data_2019')
data['applicable_date'] = pd.to_datetime(data['applicable_date'],format='%Y-%m-%d %H:%M:%S')
print(data.shape)
data.head()

(8304, 16)


,applicable_date,elec_prices,rain_d,temp_d,wdsp_d,wddir_d,rain_c,temp_c,wdsp_c,wddir_c,rain_g,temp_g,wdsp_g,wddir_g,daily_oil_price,daily_natural_gas
0,2019-01-01 00:00:00,70.28,0.0,8.8,4.626,250,0.0,8.5,2.056,240,0.0,9.9,2.056,260,59.41,2.61649
1,2019-01-01 01:00:00,65.18,0.0,8.3,4.626,250,0.0,8.5,1.542,270,0.0,9.3,2.056,240,59.41,2.61649
2,2019-01-01 02:00:00,60.91,0.0,8.4,4.112,240,0.0,8.4,1.542,280,0.0,9.2,2.570,240,59.41,2.61649
3,2019-01-01 03:00:00,65.16,0.0,8.6,4.112,250,0.0,8.5,2.570,260,0.0,9.1,2.570,250,59.41,2.61649
4,2019-01-01 04:00:00,65.16,0.0,8.4,5.140,250,0.0,8.5,2.570,270,0.0,8.8,2.056,250,59.41,2.61649


In [3]:
data['applicable_date'].min(), data['applicable_date'].max(), (data['applicable_date'].max() - data['applicable_date'].min())

(Timestamp('2019-01-01 00:00:00'),
 Timestamp('2019-12-12 23:00:00'),
 Timedelta('345 days 23:00:00'))

In [4]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8304 entries, 0 to 8303
Data columns (total 16 columns):
 #   Column             Non-Null Count  Dtype         
---  ------             --------------  -----         
 0   applicable_date    8304 non-null   datetime64[ns]
 1   elec_prices        8304 non-null   float64       
 2   rain_d             8304 non-null   float64       
 3   temp_d             8304 non-null   float64       
 4   wdsp_d             8304 non-null   float64       
 5   wddir_d            8304 non-null   int64         
 6   rain_c             8304 non-null   float64       
 7   temp_c             8304 non-null   float64       
 8   wdsp_c             8304 non-null   float64       
 9   wddir_c            8304 non-null   int64         
 10  rain_g             8304 non-null   float64       
 11  temp_g             8304 non-null   float64       
 12  wdsp_g             8304 non-null   float64       
 13  wddir_g            8304 non-null   int64         
 14  daily_oi

In [5]:
file= 'E:\CIT\MSc_AI\Research project_NIMBUS\Rework\corrected_data.xlsx'
data_1819 = pd.read_excel(file,sheet_name='data_2018_2019')
#data_1819 = pd.read_csv(file,infer_datetime_format=True,index_col=0)
data_1819['applicable_date'] = pd.to_datetime(data_1819['applicable_date'],format='%Y-%m-%d %H:%M:%S')
print(data_1819.shape)
data_1819.head()

(10514, 16)


,applicable_date,elec_prices,rain_d,temp_d,wdsp_d,wddir_d,rain_c,temp_c,wdsp_c,wddir_c,rain_g,temp_g,wdsp_g,wddir_g,daily_oil_price,daily_natural_gas
0,2018-09-30 22:00:00,71.267,0.0,6.4,5.140,290,0.0,8.2,3.084,340,0.0,5.7,1.028,300,78.89,2.78616
1,2018-09-30 23:00:00,67.212,0.0,5.4,5.140,280,0.0,8.1,3.598,320,0.0,3.9,1.542,290,78.89,2.78616
2,2018-10-01 00:00:00,60.500,0.0,5.2,5.140,280,0.0,7.7,4.112,320,0.0,6.2,1.542,270,84.94,2.79509
3,2018-10-01 01:00:00,63.682,0.0,5.2,5.654,280,0.0,7.4,3.084,320,0.0,7.2,2.056,270,84.94,2.79509
4,2018-10-01 02:00:00,71.617,0.0,4.8,4.626,270,0.0,5.3,1.542,200,0.0,6.9,2.056,280,84.94,2.79509


In [6]:
data_1819['applicable_date'].min(), data_1819['applicable_date'].max(), (data_1819['applicable_date'].max() - data_1819['applicable_date'].min())

(Timestamp('2018-09-30 22:00:00'),
 Timestamp('2019-12-12 23:00:00'),
 Timedelta('438 days 01:00:00'))

In [7]:
def normalize(x_train,x_test):
    from sklearn.preprocessing import MinMaxScaler
    scaler = MinMaxScaler().fit(x_train)
    x_train_norm = scaler.transform(x_train)
    x_test_norm = scaler.transform(x_test)
    return x_train_norm,x_test_norm

In [8]:
from sklearn.metrics import mean_absolute_error,mean_squared_error,r2_score
def mean_absolute_percentage_error(y_true, y_pred): 
    y_true, y_pred = np.array(y_true), np.array(y_pred)
    return np.mean(np.abs((y_true - y_pred) / np.mean(y_true))) * 100
def eval_metrics(y_true,y_pred):
    MAE = mean_absolute_error(y_true,y_pred)
    MSE = mean_squared_error(y_true,y_pred)
    RMSE = mean_squared_error(y_true,y_pred,squared=False)
    MAPE = mean_absolute_percentage_error(y_true,y_pred)
    R2 = r2_score(y_true, y_pred)
    print("MAE: ",round((MAE),4))
    print("MSE: ",round((MSE),4))
    print("RMSE: ",round((RMSE),4))
    print("MAPE: ",round((MAPE),4))
    print("Coefficient of determination (R-squared): ",round((R2),4))
    print('\n')
    return MAE,MSE,RMSE,MAPE,R2

In [9]:
def multi_step(data,target):

    for time_step in range(1,24):
        name = 't'+'+'+str(time_step)
        data[name] = data[target].shift(-time_step)

    return data

In [10]:
def calc_24lags(data,feature):
    if feature == 'elec_prices':
        for time_step in range(1,25):
            name = 'lag_'+str(time_step)+'_'+feature
            data[name] = data[feature].shift(time_step).bfill(axis ='rows')
    else:
        for time_step in range(1,24):
            name = 'lag_'+str(time_step)+'_'+feature
            data[name] = data[feature].shift(time_step).bfill(axis ='rows')
            
    return data

In [11]:
def remove_23rows_hour_col(data):
    data = data[data['Hour']==23]
    data= data.drop(['Hour'], axis=1)
    data = data.reset_index(drop=True)
    print(data.shape)
    return data

In [12]:
def GBM_training(X,y):

    start_time = datetime.now()

    wrapper1 = MultiOutputRegressor(GradientBoostingRegressor(max_features='sqrt', random_state=100))

    hyperparameters = {'estimator__learning_rate':[0.1,0.05,0.025,0.0125],'estimator__n_estimators':range(20,101,10),'estimator__max_depth':range(5,14,2),
                       'estimator__min_samples_split':range(100,501,100), 'estimator__max_features':range(1,7,1), 'estimator__min_samples_leaf':range(10,61,10),
                       'estimator__subsample': [0.5,0.6,0.7,0.75,0.8,0.85,0.9]}

    rsearch = RandomizedSearchCV(estimator = wrapper1, param_distributions = hyperparameters, scoring='neg_mean_absolute_error',
                                  n_jobs = -1, cv=10, return_train_score=True)

    rsearch.fit(X,y)

    end_time = datetime.now()
    print('Training Duration: {}'.format(end_time - start_time))
    #print("\n\n")
    #print(rsearch.cv_results_)
    print(rsearch.best_params_)
    print(rsearch.best_score_)
    print('\n')
    
    return rsearch.best_params_

In [13]:
def GBM_model(X,y,learning_rate,n_estimators,max_depth,min_samples_split,max_features,min_samples_leaf,subsample):
    
    learning_rate=learning_rate
    n_estimators=n_estimators
    max_depth= max_depth
    min_samples_split= min_samples_split
    max_features= max_features
    min_samples_leaf= min_samples_leaf
    subsample= subsample

    #start_time = datetime.now()

    GBM = MultiOutputRegressor(GradientBoostingRegressor(learning_rate=learning_rate, n_estimators=n_estimators, 
                                                         max_depth= max_depth,min_samples_split= min_samples_split,
                                                         max_features= max_features, min_samples_leaf= min_samples_leaf,
                                                         subsample= subsample, random_state=100))

    GBM.fit(X,y)

    #end_time = datetime.now()
    #print('Duration: {}'.format(end_time - start_time))

    return GBM

In [14]:
def XGBM_training(X,y):

    start_time = datetime.now()

    wrapper = MultiOutputRegressor(XGBRegressor(n_jobs = -1))

    hyperparameters = {'estimator__learning_rate':[0.2,0.1,0.05,0.025,0.0125],'estimator__n_estimators':range(50, 500, 50),
                       'estimator__max_depth':range(3,14,2), 'estimator__subsample':np.arange(0.1, 1.1, 0.1),
                       'estimator__colsample_bytree':np.arange(0.1, 1.1, 0.1)}

    rsearch = RandomizedSearchCV(estimator = wrapper, param_distributions = hyperparameters, scoring='neg_mean_absolute_error',
                                  n_jobs = -1, cv=10, return_train_score=True)

    rsearch.fit(X,y)

    end_time = datetime.now()
    print('Training Duration: {}'.format(end_time - start_time))
    #print("\n\n")
    #print(rsearch.cv_results_)
    print(rsearch.best_params_)
    print(rsearch.best_score_)
    print('\n')
    
    return rsearch.best_params_

In [15]:
def XGBM_model(X,y,learning_rate,n_estimators,max_depth,subsample,colsample_bytree):

    learning_rate = learning_rate
    n_estimators = n_estimators
    max_depth = max_depth
    subsample = subsample
    colsample_bytree = colsample_bytree

    #start_time = datetime.now()

    XGBM = MultiOutputRegressor(XGBRegressor(learning_rate=learning_rate, n_estimators=n_estimators, max_depth= max_depth,
                                            subsample= subsample, colsample_bytree = colsample_bytree, random_state=100))

    XGBM.fit(X,y)

    #end_time = datetime.now()
    #print('Duration: {}'.format(end_time - start_time))

    return XGBM

In [16]:
def LGBM_training(X,y):

    start_time = datetime.now()

    wrapper = MultiOutputRegressor(LGBMRegressor(random_state=100, n_jobs = -1))

    hyperparameters = {'estimator__learning_rate':[0.2,0.1,0.05,0.025,0.0125],'estimator__n_estimators':range(50, 501, 50),
                       'estimator__max_depth':range(3,14,2), 'estimator__subsample':np.arange(0.1, 1.1, 0.1),
                       'estimator__colsample_bytree':np.arange(0.1, 1.1, 0.1), 'estimator__num_leaves':[round(0.6*2**x) for x in range(3,14,2)],
                      'estimator__min_child_samples ':range(10,71,10)}

    rsearch = RandomizedSearchCV(estimator = wrapper, param_distributions = hyperparameters, scoring='neg_mean_absolute_error',
                                  n_jobs = -1, cv=10, return_train_score=True)

    rsearch.fit(X,y)

    end_time = datetime.now()
    print('Training Duration: {}'.format(end_time - start_time))
#     print("\n\n")
#     print(rsearch.cv_results_)
    print(rsearch.best_params_)
    print(rsearch.best_score_)
    print('\n')
    
    return rsearch.best_params_

In [17]:
def LGBM_model(X,y,num_leaves,min_child_samples,learning_rate,n_estimators,max_depth,subsample,colsample_bytree):

    num_leaves = num_leaves
    min_child_samples = min_child_samples
    learning_rate = learning_rate
    n_estimators = n_estimators
    max_depth= max_depth
    subsample= subsample
    colsample_bytree = colsample_bytree

    #start_time = datetime.now()

    LGB = MultiOutputRegressor(LGBMRegressor(num_leaves=num_leaves,min_child_samples = min_child_samples,learning_rate=learning_rate,n_estimators=n_estimators,
                                               max_depth= max_depth,subsample= subsample,colsample_bytree = colsample_bytree,random_state=100,
                                               n_jobs = -1))

    LGB.fit(X,y)

    #end_time = datetime.now()
    #print('Duration: {}'.format(end_time - start_time))
    
    return LGB

In [18]:
def random_splitting(data,seed):
    from sklearn.model_selection import train_test_split
    X= data[x_names]; y= data[y_names]
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.10, random_state = seed)
    #X_val, X_test, y_val, y_test = train_test_split(X_test, y_test, test_size=0.5)
    print(X_train.shape,X_test.shape)
    print(y_train.shape,y_test.shape)
    
    return X_train,X_test,y_train,y_test

In [19]:
#GBM
def testing_runs_GBM(n,data,name):
    seed = 10
    mae = []; mse = []; rmse = []; mape = []; r2 = []; models = []
    for i in range(n):
        print("Round {}: ".format(i+1))
        print('\n')
        X_train,X_test,y_train,y_test = random_splitting(data,seed)
        x_train_norm,x_test_norm = normalize(X_train,X_test)
        best_params = GBM_training(X_train,y_train)
        model = GBM_model(x_train_norm,y_train,best_params['estimator__learning_rate'],best_params['estimator__n_estimators'],
                         best_params['estimator__max_depth'],best_params['estimator__min_samples_split'],
                         best_params['estimator__max_features'],best_params['estimator__min_samples_leaf'],
                         best_params['estimator__subsample'])
        y_pred= model.predict(x_test_norm)
        MAE,MSE,RMSE,MAPE,R2 = eval_metrics(y_test,y_pred)
        mae.append(round((MAE),4))
        mse.append(round(MSE,4))
        rmse.append(round(RMSE,4))
        mape.append(round(MAPE,4))
        r2.append(round(R2,4))
        
        seed+=1
        
        models.append(model)
        
    # save the model to disk       
    model_name = 'GBM_'+name+'_model.sav'
    pickle.dump(models[np.argmin(mae)], open(model_name, 'wb'))
    
    print("Average MAE: ",round(np.mean(mae),4))
    print("Average MSE: ",round(np.mean(mse),4))
    print("Average RMSE: ", round(np.mean(rmse),4))
    print("Average MAPE: ", round(np.mean(mape),4))
    print("Average R2: ", round(np.mean(r2),4))
    
    return mae,mse,rmse,mape,r2

In [20]:
#XGBM
def testing_runs_XGBM(n,data,name):
    seed = 10
    mae = []; mse = []; rmse = []; mape = []; r2 = []; models = []
    for i in range(n):
        print("Round {}: ".format(i+1))
        print('\n')
        X_train,X_test,y_train,y_test = random_splitting(data,seed)
        x_train_norm,x_test_norm = normalize(X_train,X_test)
        best_params = XGBM_training(X_train,y_train)
        model = XGBM_model(x_train_norm,y_train,best_params['estimator__learning_rate'],best_params['estimator__n_estimators'], 
                           best_params['estimator__max_depth'],best_params['estimator__subsample'], 
                           best_params['estimator__colsample_bytree'])
        y_pred= model.predict(x_test_norm)
        MAE,MSE,RMSE,MAPE,R2 = eval_metrics(y_test,y_pred)
        mae.append(round((MAE),4))
        mse.append(round(MSE,4))
        rmse.append(round(RMSE,4))
        mape.append(round(MAPE,4))
        r2.append(round(R2,4))
        seed+=1

        models.append(model)
        
    # save the model to disk       
    model_name = 'XGBM_'+name+'_model.sav'
    pickle.dump(models[np.argmin(mae)], open(model_name, 'wb'))
                
    print("Average MAE: ",round(np.mean(mae),4))
    print("Average MSE: ",round(np.mean(mse),4))
    print("Average RMSE: ", round(np.mean(rmse),4))
    print("Average MAPE: ", round(np.mean(mape),4))
    print("Average R2: ", round(np.mean(r2),4))
    
    return mae,mse,rmse,mape,r2

In [21]:
#LGBM
def testing_runs_LGBM(n,data,name):
    seed = 10
    mae = []; mse = []; rmse = []; mape = []; r2 = []; models = []
    for i in range(n):
        print("Round {}: ".format(i+1))
        print('\n')
        X_train,X_test,y_train,y_test = random_splitting(data,seed)
        x_train_norm,x_test_norm = normalize(X_train,X_test)
        best_params = LGBM_training(X_train,y_train)
        model = LGBM_model(x_train_norm,y_train, best_params['estimator__num_leaves'], 
                           best_params['estimator__min_child_samples '], 
                           best_params['estimator__learning_rate'], best_params['estimator__n_estimators'], 
                           best_params['estimator__max_depth'], best_params['estimator__subsample'],
                           best_params['estimator__colsample_bytree'])
                         
        y_pred= model.predict(x_test_norm)
        MAE,MSE,RMSE,MAPE,R2 = eval_metrics(y_test,y_pred)
        mae.append(round((MAE),4))
        mse.append(round(MSE,4))
        rmse.append(round(RMSE,4))
        mape.append(round(MAPE,4))
        r2.append(round(R2,4))
        seed+=1

        models.append(model)
        
    # save the model to disk       
    model_name = 'LGBM_'+name+'_model.sav'
    pickle.dump(models[np.argmin(mae)], open(model_name, 'wb'))
                
                
    print("Average MAE: ",round(np.mean(mae),4))
    print("Average MSE: ",round(np.mean(mse),4))
    print("Average RMSE: ", round(np.mean(rmse),4))
    print("Average MAPE: ", round(np.mean(mape),4))
    print("Average R2: ", round(np.mean(r2),4))
    
    return mae,mse,rmse,mape,r2

In [22]:
def feature_pipeline(data,option):
    
    if option==1:

        df= data[['applicable_date','elec_prices']]
        #df['expanding_window_price'] = df['elec_prices'].expanding().mean()
        wdsp_sum= (data['wdsp_d']+data['wdsp_c']+data['wdsp_g']).values
        wdsp_avg = wdsp_sum/3
        df['wdsp_avg_log'] = np.log(wdsp_sum)
        wdsp_avg_sqrt= np.sqrt(wdsp_sum)
        df['wdsp_avg_sqrt'] = wdsp_avg_sqrt
        df['wdsp_avg'] = wdsp_avg
        #wdsp_sqr= (data['wdsp_d']*data['wdsp_d']+data['wdsp_c']*data['wdsp_c']+data['wdsp_g']*data['wdsp_g']).values
        df['wdsp_avg_sqr'] = wdsp_avg*wdsp_avg
        df['wdsp_g'] = data['wdsp_g']
        #df['wdsp_c'] = data['wdsp_c']
        df['Hour'] = df['applicable_date'].dt.hour
    
    elif option==2:

        df= data[['applicable_date','elec_prices']]
        df['expanding_window_price'] = df['elec_prices'].expanding().mean()
        wdsp_sum= (data['wdsp_d']+data['wdsp_c']+data['wdsp_g']).values
        wdsp_avg = wdsp_sum/3
        wdsp_avg_sqrt= np.sqrt(wdsp_avg)
        df['wdsp_avg_sqrt'] = wdsp_avg_sqrt
        #wdsp_sqr= (data['wdsp_d']*data['wdsp_d']+data['wdsp_c']*data['wdsp_c']+data['wdsp_g']*data['wdsp_g']).values
        df['wdsp_avg_sqr'] = wdsp_avg*wdsp_avg
        df['wdsp_g'] = data['wdsp_g']
        df['wdsp_c'] = data['wdsp_c']
        df['wdsp_avg'] = wdsp_avg
        ng = data.daily_natural_gas.values
        ng_log = np.log(ng)
        df['ng_log'] = ng_log
        df['daily_natural_gas'] = ng
        df['ng_sqr'] = ng*ng
        df['wdsp_avg_log'] = np.log(wdsp_avg)
        df['ng_sqrt'] = np.sqrt(ng)
        df['wdsp_d'] = data['wdsp_d']
        df['Hour'] = df['applicable_date'].dt.hour
    
    return df

In [23]:
def to_excel(mae,mse,rmse,mape,r2):
    result_items = pd.DataFrame(columns=["mae", "mse", "rmse", "mape", "r2"])
    result_items['mae'] = mae
    result_items['mse'] = mse
    result_items['rmse'] = rmse
    result_items['mape'] = mape
    result_items['r2'] = r2
    path = "E:/CIT/MSc_AI/Research project_NIMBUS/Rework/output.xlsx"
    writer = pd.ExcelWriter(path)
    result_items.to_excel(writer, index=False)
    writer.close()

### Testing on 2018-2019 data

In [24]:
def feature_pipeline_2(data):
    

    df= data[['applicable_date','elec_prices']]
    wdsp_sum= (data['wdsp_d']+data['wdsp_c']+data['wdsp_g']).values
    wdsp_avg = wdsp_sum/3
    df['wdsp_avg_log'] = np.log(wdsp_sum)
    wdsp_avg_sqrt= np.sqrt(wdsp_sum)
    df['wdsp_avg_sqrt'] = wdsp_avg_sqrt
    df['wdsp_avg'] = wdsp_avg
    df['wdsp_avg_sqr'] = wdsp_avg*wdsp_avg
    df['wdsp_g'] = data['wdsp_g']
    df['Hour'] = df['applicable_date'].dt.hour
    
    return df

In [25]:
data_test = data_1819.copy()
data_test = feature_pipeline_2(data_test)
print(data_test.shape)
print(data_test.columns)
data_test.head()

(10514, 8)
Index(['applicable_date', 'elec_prices', 'wdsp_avg_log', 'wdsp_avg_sqrt',
       'wdsp_avg', 'wdsp_avg_sqr', 'wdsp_g', 'Hour'],
      dtype='object')


,applicable_date,elec_prices,wdsp_avg_log,wdsp_avg_sqrt,wdsp_avg,wdsp_avg_sqr,wdsp_g,Hour
0,2018-09-30 22:00:00,71.267,2.224840,3.041710,3.084000,9.511056,1.028,22
1,2018-09-30 23:00:00,67.212,2.330200,3.206244,3.426667,11.742044,1.542,23
2,2018-10-01 00:00:00,60.500,2.378990,3.285422,3.598000,12.945604,1.542,0
3,2018-10-01 01:00:00,63.682,2.378990,3.285422,3.598000,12.945604,2.056,1
4,2018-10-01 02:00:00,71.617,2.107057,2.867752,2.741333,7.514908,2.056,2


In [26]:
data_test = multi_step(data_test,'elec_prices')

data_test = calc_24lags(data_test,'elec_prices')
data_test = calc_24lags(data_test,'wdsp_avg_log')
data_test = calc_24lags(data_test,'wdsp_avg_sqrt')
data_test = calc_24lags(data_test,'wdsp_avg')
data_test = calc_24lags(data_test,'wdsp_avg_sqr')
data_test = calc_24lags(data_test,'wdsp_g')

In [27]:
print(data_test.shape)
print(data_test.columns)
data_test.head()

(10514, 170)
Index(['applicable_date', 'elec_prices', 'wdsp_avg_log', 'wdsp_avg_sqrt',
       'wdsp_avg', 'wdsp_avg_sqr', 'wdsp_g', 'Hour', 't+1', 't+2',
       ...
       'lag_14_wdsp_g', 'lag_15_wdsp_g', 'lag_16_wdsp_g', 'lag_17_wdsp_g',
       'lag_18_wdsp_g', 'lag_19_wdsp_g', 'lag_20_wdsp_g', 'lag_21_wdsp_g',
       'lag_22_wdsp_g', 'lag_23_wdsp_g'],
      dtype='object', length=170)


,applicable_date,elec_prices,wdsp_avg_log,wdsp_avg_sqrt,wdsp_avg,wdsp_avg_sqr,wdsp_g,Hour,t+1,t+2,...,lag_14_wdsp_g,lag_15_wdsp_g,lag_16_wdsp_g,lag_17_wdsp_g,lag_18_wdsp_g,lag_19_wdsp_g,lag_20_wdsp_g,lag_21_wdsp_g,lag_22_wdsp_g,lag_23_wdsp_g
0,2018-09-30 22:00:00,71.267,2.224840,3.041710,3.084000,9.511056,1.028,22,67.212,60.500,...,1.028,1.028,1.028,1.028,1.028,1.028,1.028,1.028,1.028,1.028
1,2018-09-30 23:00:00,67.212,2.330200,3.206244,3.426667,11.742044,1.542,23,60.500,63.682,...,1.028,1.028,1.028,1.028,1.028,1.028,1.028,1.028,1.028,1.028
2,2018-10-01 00:00:00,60.500,2.378990,3.285422,3.598000,12.945604,1.542,0,63.682,71.617,...,1.028,1.028,1.028,1.028,1.028,1.028,1.028,1.028,1.028,1.028
3,2018-10-01 01:00:00,63.682,2.378990,3.285422,3.598000,12.945604,2.056,1,71.617,72.855,...,1.028,1.028,1.028,1.028,1.028,1.028,1.028,1.028,1.028,1.028
4,2018-10-01 02:00:00,71.617,2.107057,2.867752,2.741333,7.514908,2.056,2,72.855,76.937,...,1.028,1.028,1.028,1.028,1.028,1.028,1.028,1.028,1.028,1.028


In [28]:
#Dropping rows with Nan's
data_test = data_test.dropna(axis=0)
print(data_test.shape)
data_test.isnull().values.any()

(10491, 170)


False

In [29]:
data_test = remove_23rows_hour_col(data_test)

(438, 169)


In [30]:
#selecting columns for X & y
cols = data_test.columns.to_list()
x_names = cols[2:7]
lags = cols[30:]
x_names.extend(lags)
y_names = [cols[1]]
time_steps = cols[7:30]
y_names.extend(time_steps)
print(x_names)
print(y_names)

['wdsp_avg_log', 'wdsp_avg_sqrt', 'wdsp_avg', 'wdsp_avg_sqr', 'wdsp_g', 'lag_1_elec_prices', 'lag_2_elec_prices', 'lag_3_elec_prices', 'lag_4_elec_prices', 'lag_5_elec_prices', 'lag_6_elec_prices', 'lag_7_elec_prices', 'lag_8_elec_prices', 'lag_9_elec_prices', 'lag_10_elec_prices', 'lag_11_elec_prices', 'lag_12_elec_prices', 'lag_13_elec_prices', 'lag_14_elec_prices', 'lag_15_elec_prices', 'lag_16_elec_prices', 'lag_17_elec_prices', 'lag_18_elec_prices', 'lag_19_elec_prices', 'lag_20_elec_prices', 'lag_21_elec_prices', 'lag_22_elec_prices', 'lag_23_elec_prices', 'lag_24_elec_prices', 'lag_1_wdsp_avg_log', 'lag_2_wdsp_avg_log', 'lag_3_wdsp_avg_log', 'lag_4_wdsp_avg_log', 'lag_5_wdsp_avg_log', 'lag_6_wdsp_avg_log', 'lag_7_wdsp_avg_log', 'lag_8_wdsp_avg_log', 'lag_9_wdsp_avg_log', 'lag_10_wdsp_avg_log', 'lag_11_wdsp_avg_log', 'lag_12_wdsp_avg_log', 'lag_13_wdsp_avg_log', 'lag_14_wdsp_avg_log', 'lag_15_wdsp_avg_log', 'lag_16_wdsp_avg_log', 'lag_17_wdsp_avg_log', 'lag_18_wdsp_avg_log', 'lag

In [69]:
#GBM
start_time = datetime.now()
mae1,mse1,rmse1,mape1,r21 = testing_runs_GBM(10,data_test,'top5')
end_time = datetime.now()
print('Process Duration: {}'.format(end_time - start_time))

Round 1: 


(394, 144) (44, 144)
(394, 24) (44, 24)
Training Duration: 0:00:09.236802
{'estimator__subsample': 0.6, 'estimator__n_estimators': 90, 'estimator__min_samples_split': 200, 'estimator__min_samples_leaf': 40, 'estimator__max_features': 4, 'estimator__max_depth': 7, 'estimator__learning_rate': 0.0125}
-13.171397164924562


MAE:  13.4662
MSE:  431.5878
RMSE:  20.7747
MAPE:  23.9905
Coefficient of determination (R-squared):  0.2163


Round 2: 


(394, 144) (44, 144)
(394, 24) (44, 24)
Training Duration: 0:00:09.886765
{'estimator__subsample': 0.6, 'estimator__n_estimators': 80, 'estimator__min_samples_split': 100, 'estimator__min_samples_leaf': 10, 'estimator__max_features': 6, 'estimator__max_depth': 9, 'estimator__learning_rate': 0.05}
-10.591475389236649


MAE:  8.4912
MSE:  148.3441
RMSE:  12.1797
MAPE:  16.1986
Coefficient of determination (R-squared):  0.5131


Round 3: 


(394, 144) (44, 144)
(394, 24) (44, 24)
Training Duration: 0:00:08.290015
{'estimator__subsample': 0.

In [70]:
to_excel(mae1,mse1,rmse1,mape1,r21)

In [71]:
#XGBM
start_time = datetime.now()
mae2,mse2,rmse2,mape2,r22 = testing_runs_XGBM(10,data_test,'top5')
end_time = datetime.now()
print('Process Duration: {}'.format(end_time - start_time))

Round 1: 


(394, 144) (44, 144)
(394, 24) (44, 24)
Training Duration: 0:04:04.500450
{'estimator__subsample': 0.8, 'estimator__n_estimators': 250, 'estimator__max_depth': 11, 'estimator__learning_rate': 0.0125, 'estimator__colsample_bytree': 0.6}
-10.097563040790288


MAE:  11.0909
MSE:  387.255
RMSE:  19.6788
MAPE:  19.7587
Coefficient of determination (R-squared):  0.38


Round 2: 


(394, 144) (44, 144)
(394, 24) (44, 24)
Training Duration: 0:05:28.562604
{'estimator__subsample': 0.4, 'estimator__n_estimators': 450, 'estimator__max_depth': 9, 'estimator__learning_rate': 0.0125, 'estimator__colsample_bytree': 0.1}
-10.484225190084276


MAE:  8.4775
MSE:  142.8895
RMSE:  11.9536
MAPE:  16.1725
Coefficient of determination (R-squared):  0.5271


Round 3: 


(394, 144) (44, 144)
(394, 24) (44, 24)
Training Duration: 0:02:46.926267
{'estimator__subsample': 0.5, 'estimator__n_estimators': 150, 'estimator__max_depth': 9, 'estimator__learning_rate': 0.05, 'estimator__colsample_bytree': 0.8

In [72]:
to_excel(mae2,mse2,rmse2,mape2,r22)

In [59]:
#LGBM
start_time = datetime.now()
mae3,mse3,rmse3,mape3,r23 = testing_runs_LGBM(10,data_test,'top5')
end_time = datetime.now()
print('Process Duration: {}'.format(end_time - start_time))

Round 1: 


(394, 144) (44, 144)
(394, 24) (44, 24)
Training Duration: 0:02:25.352348
{'estimator__subsample': 0.5, 'estimator__num_leaves': 1229, 'estimator__n_estimators': 300, 'estimator__min_child_samples ': 10, 'estimator__max_depth': 7, 'estimator__learning_rate': 0.0125, 'estimator__colsample_bytree': 0.5}
-10.058542026096683


MAE:  10.9381
MSE:  384.2509
RMSE:  19.6023
MAPE:  19.4865
Coefficient of determination (R-squared):  0.4093


Round 2: 


(394, 144) (44, 144)
(394, 24) (44, 24)
[LightGBM] [Warning] min_child_samples is set=20, min_child_samples= will be ignored. Current value: min_child_samples=20
[LightGBM] [Warning] Unknown parameter: 50
Training Duration: 0:01:37.891303
{'estimator__subsample': 0.1, 'estimator__num_leaves': 307, 'estimator__n_estimators': 300, 'estimator__min_child_samples ': 50, 'estimator__max_depth': 9, 'estimator__learning_rate': 0.0125, 'estimator__colsample_bytree': 0.6}
-10.516210803942512


MAE:  8.0463
MSE:  132.4267
RMSE:  11.5077
MAPE:  1

In [73]:
check = pickle.load(open('XGBM_top5_model.sav', 'rb'))
X_train,X_test,y_train,y_test = random_splitting(data_test,11)
x_train_norm,x_test_norm = normalize(X_train,X_test)
y_pred= check.predict(x_test_norm)
MAE,MSE,RMSE,MAPE,R2 = eval_metrics(y_test,y_pred)

(394, 144) (44, 144)
(394, 24) (44, 24)
MAE:  8.4775
MSE:  142.8895
RMSE:  11.9536
MAPE:  16.1725
Coefficient of determination (R-squared):  0.5271




In [61]:
to_excel(mae3,mse3,rmse3,mape3,r23)

### Testing for 30 rounds

In [74]:
#GBM
start_time = datetime.now()
mae4,mse4,rmse4,mape4,r24 = testing_runs_GBM(30,data_test,'top5_30rounds')
end_time = datetime.now()
print('Process Duration: {}'.format(end_time - start_time))

Round 1: 


(394, 144) (44, 144)
(394, 24) (44, 24)
Training Duration: 0:00:10.929656
{'estimator__subsample': 0.5, 'estimator__n_estimators': 100, 'estimator__min_samples_split': 100, 'estimator__min_samples_leaf': 40, 'estimator__max_features': 5, 'estimator__max_depth': 5, 'estimator__learning_rate': 0.05}
-10.58610890243995


MAE:  10.9278
MSE:  334.58
RMSE:  18.2915
MAPE:  19.4682
Coefficient of determination (R-squared):  0.438


Round 2: 


(394, 144) (44, 144)
(394, 24) (44, 24)
Training Duration: 0:00:07.575266
{'estimator__subsample': 0.9, 'estimator__n_estimators': 80, 'estimator__min_samples_split': 200, 'estimator__min_samples_leaf': 60, 'estimator__max_features': 6, 'estimator__max_depth': 11, 'estimator__learning_rate': 0.05}
-10.975409738563743


MAE:  8.6126
MSE:  150.415
RMSE:  12.2644
MAPE:  16.4302
Coefficient of determination (R-squared):  0.4992


Round 3: 


(394, 144) (44, 144)
(394, 24) (44, 24)
Training Duration: 0:00:07.590767
{'estimator__subsample': 0.85, '

MAE:  10.6101
MSE:  302.1964
RMSE:  17.3838
MAPE:  18.9041
Coefficient of determination (R-squared):  0.463


Round 20: 


(394, 144) (44, 144)
(394, 24) (44, 24)
Training Duration: 0:00:09.117630
{'estimator__subsample': 0.85, 'estimator__n_estimators': 70, 'estimator__min_samples_split': 100, 'estimator__min_samples_leaf': 50, 'estimator__max_features': 6, 'estimator__max_depth': 9, 'estimator__learning_rate': 0.1}
-10.503292938437749


MAE:  10.5639
MSE:  270.9125
RMSE:  16.4594
MAPE:  18.8461
Coefficient of determination (R-squared):  0.5425


Round 21: 


(394, 144) (44, 144)
(394, 24) (44, 24)
Training Duration: 0:00:08.660343
{'estimator__subsample': 0.9, 'estimator__n_estimators': 80, 'estimator__min_samples_split': 100, 'estimator__min_samples_leaf': 60, 'estimator__max_features': 5, 'estimator__max_depth': 5, 'estimator__learning_rate': 0.0125}
-12.554111165988108


MAE:  10.75
MSE:  247.3948
RMSE:  15.7288
MAPE:  20.7572
Coefficient of determination (R-squared):  0.2869


Ro

In [75]:
to_excel(mae4,mse4,rmse4,mape4,r24)

In [33]:
#XGBM
start_time = datetime.now()
mae5,mse5,rmse5,mape5,r25 = testing_runs_XGBM(30,data_test,'top5_30rounds')
end_time = datetime.now()
print('Process Duration: {}'.format(end_time - start_time))

Round 1: 


(394, 144) (44, 144)
(394, 24) (44, 24)
Training Duration: 0:03:39.109940
{'estimator__subsample': 0.8, 'estimator__n_estimators': 200, 'estimator__max_depth': 5, 'estimator__learning_rate': 0.025, 'estimator__colsample_bytree': 0.4}
-9.828978216058877


MAE:  10.7308
MSE:  364.8949
RMSE:  19.1022
MAPE:  19.1173
Coefficient of determination (R-squared):  0.4223


Round 2: 


(394, 144) (44, 144)
(394, 24) (44, 24)
Training Duration: 0:02:11.739143
{'estimator__subsample': 0.7000000000000001, 'estimator__n_estimators': 400, 'estimator__max_depth': 7, 'estimator__learning_rate': 0.0125, 'estimator__colsample_bytree': 0.2}
-10.396724485404572


MAE:  8.2826
MSE:  139.6025
RMSE:  11.8154
MAPE:  15.8006
Coefficient of determination (R-squared):  0.5439


Round 3: 


(394, 144) (44, 144)
(394, 24) (44, 24)
Training Duration: 0:03:28.158629
{'estimator__subsample': 0.30000000000000004, 'estimator__n_estimators': 350, 'estimator__max_depth': 11, 'estimator__learning_rate': 0.0125, 

MAE:  9.5616
MSE:  193.162
RMSE:  13.8983
MAPE:  17.6033
Coefficient of determination (R-squared):  0.5105


Round 23: 


(394, 144) (44, 144)
(394, 24) (44, 24)
Training Duration: 0:04:47.599082
{'estimator__subsample': 0.6, 'estimator__n_estimators': 450, 'estimator__max_depth': 11, 'estimator__learning_rate': 0.0125, 'estimator__colsample_bytree': 0.6}
-10.365833456200335


MAE:  8.6647
MSE:  222.5824
RMSE:  14.9192
MAPE:  16.2225
Coefficient of determination (R-squared):  0.5764


Round 24: 


(394, 144) (44, 144)
(394, 24) (44, 24)
Training Duration: 0:03:00.294934
{'estimator__subsample': 0.7000000000000001, 'estimator__n_estimators': 400, 'estimator__max_depth': 3, 'estimator__learning_rate': 0.05, 'estimator__colsample_bytree': 0.8}
-10.425683035262557


MAE:  10.2368
MSE:  255.9694
RMSE:  15.999
MAPE:  19.3409
Coefficient of determination (R-squared):  0.3666


Round 25: 


(394, 144) (44, 144)
(394, 24) (44, 24)
Training Duration: 0:05:02.276021
{'estimator__subsample': 0.6, 

In [34]:
to_excel(mae5,mse5,rmse5,mape5,r25)

In [31]:
#LGBM
start_time = datetime.now()
mae6,mse6,rmse6,mape6,r26 = testing_runs_LGBM(30,data_test,'top5_30rounds')
end_time = datetime.now()
print('Process Duration: {}'.format(end_time - start_time))

Round 1: 


(394, 144) (44, 144)
(394, 24) (44, 24)
[LightGBM] [Warning] min_child_samples is set=20, min_child_samples= will be ignored. Current value: min_child_samples=20
[LightGBM] [Warning] Unknown parameter: 70
Training Duration: 0:01:51.288210
{'estimator__subsample': 0.1, 'estimator__num_leaves': 5, 'estimator__n_estimators': 500, 'estimator__min_child_samples ': 70, 'estimator__max_depth': 9, 'estimator__learning_rate': 0.0125, 'estimator__colsample_bytree': 0.7000000000000001}
-10.134230726177998


MAE:  11.146
MSE:  348.8941
RMSE:  18.6787
MAPE:  19.857
Coefficient of determination (R-squared):  0.4283


Round 2: 


(394, 144) (44, 144)
(394, 24) (44, 24)
[LightGBM] [Warning] min_child_samples is set=20, min_child_samples= will be ignored. Current value: min_child_samples=20
[LightGBM] [Warning] Unknown parameter: 20
Training Duration: 0:01:44.949844
{'estimator__subsample': 0.2, 'estimator__num_leaves': 19, 'estimator__n_estimators': 350, 'estimator__min_child_samples ': 20

Training Duration: 0:01:33.360811
{'estimator__subsample': 0.2, 'estimator__num_leaves': 307, 'estimator__n_estimators': 350, 'estimator__min_child_samples ': 30, 'estimator__max_depth': 3, 'estimator__learning_rate': 0.0125, 'estimator__colsample_bytree': 0.1}
-10.006917033928191


MAE:  12.05
MSE:  356.2364
RMSE:  18.8742
MAPE:  22.6187
Coefficient of determination (R-squared):  0.4108


Round 15: 


(394, 144) (44, 144)
(394, 24) (44, 24)
[LightGBM] [Warning] min_child_samples is set=20, min_child_samples= will be ignored. Current value: min_child_samples=20
[LightGBM] [Warning] Unknown parameter: 40
Training Duration: 0:01:00.491414
{'estimator__subsample': 0.7000000000000001, 'estimator__num_leaves': 5, 'estimator__n_estimators': 150, 'estimator__min_child_samples ': 40, 'estimator__max_depth': 11, 'estimator__learning_rate': 0.05, 'estimator__colsample_bytree': 0.1}
-10.133776547429203


MAE:  11.4263
MSE:  325.8602
RMSE:  18.0516
MAPE:  21.143
Coefficient of determination (R-squ

MAE:  9.2937
MSE:  217.7055
RMSE:  14.7548
MAPE:  16.979
Coefficient of determination (R-squared):  0.5471


Round 28: 


(394, 144) (44, 144)
(394, 24) (44, 24)
[LightGBM] [Warning] min_child_samples is set=20, min_child_samples= will be ignored. Current value: min_child_samples=20
[LightGBM] [Warning] Unknown parameter: 70
Training Duration: 0:01:04.569701
{'estimator__subsample': 0.8, 'estimator__num_leaves': 19, 'estimator__n_estimators': 50, 'estimator__min_child_samples ': 70, 'estimator__max_depth': 9, 'estimator__learning_rate': 0.05, 'estimator__colsample_bytree': 0.4}
-10.306259194641063


MAE:  10.4247
MSE:  243.5805
RMSE:  15.6071
MAPE:  19.3118
Coefficient of determination (R-squared):  0.2948


Round 29: 


(394, 144) (44, 144)
(394, 24) (44, 24)
[LightGBM] [Warning] min_child_samples is set=20, min_child_samples= will be ignored. Current value: min_child_samples=20
[LightGBM] [Warning] Unknown parameter: 60
Training Duration: 0:01:11.641075
{'estimator__subsample': 0.1, 

In [32]:
to_excel(mae6,mse6,rmse6,mape6,r26)

<h1 style="text-align:center;font-size:40px;" > All 12 imp features</h1>

### Testing on 2018-2019 data

In [35]:
data_test = data_1819.copy()
data_test = feature_pipeline(data_test,2)
data_test = multi_step(data_test,'elec_prices')
data_test = calc_24lags(data_test,'elec_prices')
data_test = calc_24lags(data_test,'wdsp_avg_sqrt')
data_test = calc_24lags(data_test,'wdsp_avg_sqr')
data_test = calc_24lags(data_test,'wdsp_g')
data_test = calc_24lags(data_test,'wdsp_c')
data_test = calc_24lags(data_test,'wdsp_avg')
data_test = calc_24lags(data_test,'ng_log')
data_test = calc_24lags(data_test,'daily_natural_gas')
data_test = calc_24lags(data_test,'ng_sqr')
data_test = calc_24lags(data_test,'wdsp_avg_log')
data_test = calc_24lags(data_test,'ng_sqrt')
data_test = calc_24lags(data_test,'wdsp_d')
print(data_test.shape)
print(data_test.columns)
data_test.head()

(10514, 315)
Index(['applicable_date', 'elec_prices', 'expanding_window_price',
       'wdsp_avg_sqrt', 'wdsp_avg_sqr', 'wdsp_g', 'wdsp_c', 'wdsp_avg',
       'ng_log', 'daily_natural_gas',
       ...
       'lag_14_wdsp_d', 'lag_15_wdsp_d', 'lag_16_wdsp_d', 'lag_17_wdsp_d',
       'lag_18_wdsp_d', 'lag_19_wdsp_d', 'lag_20_wdsp_d', 'lag_21_wdsp_d',
       'lag_22_wdsp_d', 'lag_23_wdsp_d'],
      dtype='object', length=315)


,applicable_date,elec_prices,expanding_window_price,wdsp_avg_sqrt,wdsp_avg_sqr,wdsp_g,wdsp_c,wdsp_avg,ng_log,daily_natural_gas,...,lag_14_wdsp_d,lag_15_wdsp_d,lag_16_wdsp_d,lag_17_wdsp_d,lag_18_wdsp_d,lag_19_wdsp_d,lag_20_wdsp_d,lag_21_wdsp_d,lag_22_wdsp_d,lag_23_wdsp_d
0,2018-09-30 22:00:00,71.267,71.267000,1.756132,9.511056,1.028,3.084,3.084000,1.024664,2.78616,...,5.14,5.14,5.14,5.14,5.14,5.14,5.14,5.14,5.14,5.14
1,2018-09-30 23:00:00,67.212,69.239500,1.851126,11.742044,1.542,3.598,3.426667,1.024664,2.78616,...,5.14,5.14,5.14,5.14,5.14,5.14,5.14,5.14,5.14,5.14
2,2018-10-01 00:00:00,60.500,66.326333,1.896839,12.945604,1.542,4.112,3.598000,1.027864,2.79509,...,5.14,5.14,5.14,5.14,5.14,5.14,5.14,5.14,5.14,5.14
3,2018-10-01 01:00:00,63.682,65.665250,1.896839,12.945604,2.056,3.084,3.598000,1.027864,2.79509,...,5.14,5.14,5.14,5.14,5.14,5.14,5.14,5.14,5.14,5.14
4,2018-10-01 02:00:00,71.617,66.855600,1.655697,7.514908,2.056,1.542,2.741333,1.027864,2.79509,...,5.14,5.14,5.14,5.14,5.14,5.14,5.14,5.14,5.14,5.14


In [36]:
#Dropping rows with Nan's
data_test = data_test.dropna(axis=0)
print(data_test.shape)
data_test.isnull().values.any()

(10491, 315)


False

In [37]:
data_test = remove_23rows_hour_col(data_test)
data_test.columns

(438, 314)


Index(['applicable_date', 'elec_prices', 'expanding_window_price',
       'wdsp_avg_sqrt', 'wdsp_avg_sqr', 'wdsp_g', 'wdsp_c', 'wdsp_avg',
       'ng_log', 'daily_natural_gas',
       ...
       'lag_14_wdsp_d', 'lag_15_wdsp_d', 'lag_16_wdsp_d', 'lag_17_wdsp_d',
       'lag_18_wdsp_d', 'lag_19_wdsp_d', 'lag_20_wdsp_d', 'lag_21_wdsp_d',
       'lag_22_wdsp_d', 'lag_23_wdsp_d'],
      dtype='object', length=314)

In [38]:
data_test.head()

,applicable_date,elec_prices,expanding_window_price,wdsp_avg_sqrt,wdsp_avg_sqr,wdsp_g,wdsp_c,wdsp_avg,ng_log,daily_natural_gas,...,lag_14_wdsp_d,lag_15_wdsp_d,lag_16_wdsp_d,lag_17_wdsp_d,lag_18_wdsp_d,lag_19_wdsp_d,lag_20_wdsp_d,lag_21_wdsp_d,lag_22_wdsp_d,lag_23_wdsp_d
0,2018-09-30 23:00:00,67.212,69.239500,1.851126,11.742044,1.542,3.598,3.426667,1.024664,2.78616,...,5.140,5.140,5.140,5.140,5.140,5.140,5.140,5.140,5.140,5.140
1,2018-10-01 23:00:00,56.940,80.490308,2.304633,28.210262,3.598,5.140,5.311333,1.027864,2.79509,...,5.140,5.654,5.654,4.626,3.084,2.570,3.084,4.626,5.654,5.140
2,2018-10-02 23:00:00,62.290,75.592040,1.985111,15.528854,3.084,3.084,3.940667,1.031054,2.80402,...,7.196,6.168,6.168,6.682,8.224,8.224,8.224,6.682,6.168,6.168
3,2018-10-03 23:00:00,60.350,76.635622,1.603122,6.604900,3.084,2.570,2.570000,1.068558,2.91118,...,4.112,3.598,4.112,4.112,4.112,4.112,4.626,5.140,4.112,4.626
4,2018-10-04 23:00:00,70.000,76.759357,1.603122,6.604900,2.056,3.598,2.570000,1.107661,3.02727,...,3.598,3.084,3.598,3.598,3.598,2.056,1.028,2.056,3.084,2.056


In [39]:
#selecting columns for X & y
cols = data_test.columns.to_list()
x_names = cols[2:14]
lags = cols[37:]
x_names.extend(lags)
y_names = [cols[1]]
time_steps = cols[14:37]
y_names.extend(time_steps)
print(x_names)
print(y_names)

['expanding_window_price', 'wdsp_avg_sqrt', 'wdsp_avg_sqr', 'wdsp_g', 'wdsp_c', 'wdsp_avg', 'ng_log', 'daily_natural_gas', 'ng_sqr', 'wdsp_avg_log', 'ng_sqrt', 'wdsp_d', 'lag_1_elec_prices', 'lag_2_elec_prices', 'lag_3_elec_prices', 'lag_4_elec_prices', 'lag_5_elec_prices', 'lag_6_elec_prices', 'lag_7_elec_prices', 'lag_8_elec_prices', 'lag_9_elec_prices', 'lag_10_elec_prices', 'lag_11_elec_prices', 'lag_12_elec_prices', 'lag_13_elec_prices', 'lag_14_elec_prices', 'lag_15_elec_prices', 'lag_16_elec_prices', 'lag_17_elec_prices', 'lag_18_elec_prices', 'lag_19_elec_prices', 'lag_20_elec_prices', 'lag_21_elec_prices', 'lag_22_elec_prices', 'lag_23_elec_prices', 'lag_24_elec_prices', 'lag_1_wdsp_avg_sqrt', 'lag_2_wdsp_avg_sqrt', 'lag_3_wdsp_avg_sqrt', 'lag_4_wdsp_avg_sqrt', 'lag_5_wdsp_avg_sqrt', 'lag_6_wdsp_avg_sqrt', 'lag_7_wdsp_avg_sqrt', 'lag_8_wdsp_avg_sqrt', 'lag_9_wdsp_avg_sqrt', 'lag_10_wdsp_avg_sqrt', 'lag_11_wdsp_avg_sqrt', 'lag_12_wdsp_avg_sqrt', 'lag_13_wdsp_avg_sqrt', 'lag_14_

In [41]:
len(x_names)


289

In [42]:
len(y_names)

24

### 10 runs

In [43]:
#GBM
start_time = datetime.now()
mae7,mse7,rmse7,mape7,r27 = testing_runs_GBM(10,data_test,'top12_10_rounds')
end_time = datetime.now()
print('Process Duration: {}'.format(end_time - start_time))

Round 1: 


(394, 289) (44, 289)
(394, 24) (44, 24)
Training Duration: 0:00:21.812176
{'estimator__subsample': 0.85, 'estimator__n_estimators': 90, 'estimator__min_samples_split': 300, 'estimator__min_samples_leaf': 60, 'estimator__max_features': 6, 'estimator__max_depth': 13, 'estimator__learning_rate': 0.025}
-11.464505769657428


MAE:  11.9637
MSE:  387.3307
RMSE:  19.6807
MAPE:  21.3137
Coefficient of determination (R-squared):  0.3244


Round 2: 


(394, 289) (44, 289)
(394, 24) (44, 24)
Training Duration: 0:00:11.047311
{'estimator__subsample': 0.7, 'estimator__n_estimators': 60, 'estimator__min_samples_split': 100, 'estimator__min_samples_leaf': 30, 'estimator__max_features': 5, 'estimator__max_depth': 9, 'estimator__learning_rate': 0.05}
-11.043691912020472


MAE:  8.5885
MSE:  150.9261
RMSE:  12.2852
MAPE:  16.3842
Coefficient of determination (R-squared):  0.5135


Round 3: 


(394, 289) (44, 289)
(394, 24) (44, 24)
Training Duration: 0:00:12.911871
{'estimator__subsample': 0

In [44]:
to_excel(mae7,mse7,rmse7,mape7,r27)

In [45]:
#XGBM
start_time = datetime.now()
mae8,mse8,rmse8,mape8,r28 = testing_runs_XGBM(10,data_test,'top12_10_rounds')
end_time = datetime.now()
print('Process Duration: {}'.format(end_time - start_time))

Round 1: 


(394, 289) (44, 289)
(394, 24) (44, 24)
Training Duration: 0:12:34.031482
{'estimator__subsample': 0.6, 'estimator__n_estimators': 350, 'estimator__max_depth': 13, 'estimator__learning_rate': 0.0125, 'estimator__colsample_bytree': 0.9}
-9.824594119181459


MAE:  10.4345
MSE:  352.7981
RMSE:  18.7829
MAPE:  18.5893
Coefficient of determination (R-squared):  0.4231


Round 2: 


(394, 289) (44, 289)
(394, 24) (44, 24)
Training Duration: 0:11:43.583288
{'estimator__subsample': 0.30000000000000004, 'estimator__n_estimators': 400, 'estimator__max_depth': 7, 'estimator__learning_rate': 0.0125, 'estimator__colsample_bytree': 0.30000000000000004}
-10.184108614055871


MAE:  8.0294
MSE:  139.8039
RMSE:  11.8239
MAPE:  15.3176
Coefficient of determination (R-squared):  0.5514


Round 3: 


(394, 289) (44, 289)
(394, 24) (44, 24)
Training Duration: 0:12:55.764919
{'estimator__subsample': 0.4, 'estimator__n_estimators': 350, 'estimator__max_depth': 7, 'estimator__learning_rate': 0.0125

In [46]:
to_excel(mae8,mse8,rmse8,mape8,r28)

In [47]:
#LGBM
start_time = datetime.now()
mae9,mse9,rmse9,mape9,r29 = testing_runs_LGBM(10,data_test,'top12_10_rounds')
end_time = datetime.now()
print('Process Duration: {}'.format(end_time - start_time))

Round 1: 


(394, 289) (44, 289)
(394, 24) (44, 24)
[LightGBM] [Warning] min_child_samples is set=20, min_child_samples= will be ignored. Current value: min_child_samples=20
[LightGBM] [Warning] Unknown parameter: 10
Training Duration: 0:02:57.176936
{'estimator__subsample': 0.7000000000000001, 'estimator__num_leaves': 4915, 'estimator__n_estimators': 150, 'estimator__min_child_samples ': 10, 'estimator__max_depth': 11, 'estimator__learning_rate': 0.025, 'estimator__colsample_bytree': 0.6}
-10.025559301462428


MAE:  10.7389
MSE:  366.377
RMSE:  19.141
MAPE:  19.1317
Coefficient of determination (R-squared):  0.4226


Round 2: 


(394, 289) (44, 289)
(394, 24) (44, 24)
[LightGBM] [Warning] min_child_samples is set=20, min_child_samples= will be ignored. Current value: min_child_samples=20
[LightGBM] [Warning] Unknown parameter: 70
Training Duration: 0:03:12.273584
{'estimator__subsample': 1.0, 'estimator__num_leaves': 77, 'estimator__n_estimators': 250, 'estimator__min_child_samples ':

In [48]:
to_excel(mae9,mse9,rmse9,mape9,r29)

### 30 runs

In [49]:
#GBM
start_time = datetime.now()
mae10,mse10,rmse10,mape10,r210 = testing_runs_GBM(30,data_test,'top12_30_rounds')
end_time = datetime.now()
print('Process Duration: {}'.format(end_time - start_time))

Round 1: 


(394, 289) (44, 289)
(394, 24) (44, 24)
Training Duration: 0:00:22.671295
{'estimator__subsample': 0.8, 'estimator__n_estimators': 90, 'estimator__min_samples_split': 100, 'estimator__min_samples_leaf': 40, 'estimator__max_features': 5, 'estimator__max_depth': 13, 'estimator__learning_rate': 0.05}
-10.675896837937442


MAE:  11.0901
MSE:  355.5161
RMSE:  18.8551
MAPE:  19.7573
Coefficient of determination (R-squared):  0.4004


Round 2: 


(394, 289) (44, 289)
(394, 24) (44, 24)
Training Duration: 0:00:13.703861
{'estimator__subsample': 0.7, 'estimator__n_estimators': 60, 'estimator__min_samples_split': 100, 'estimator__min_samples_leaf': 20, 'estimator__max_features': 6, 'estimator__max_depth': 11, 'estimator__learning_rate': 0.05}
-10.871248919917765


MAE:  8.4033
MSE:  144.614
RMSE:  12.0256
MAPE:  16.0307
Coefficient of determination (R-squared):  0.538


Round 3: 


(394, 289) (44, 289)
(394, 24) (44, 24)
Training Duration: 0:00:17.547210
{'estimator__subsample': 0.6,

MAE:  10.3415
MSE:  295.3869
RMSE:  17.1868
MAPE:  18.4255
Coefficient of determination (R-squared):  0.481


Round 20: 


(394, 289) (44, 289)
(394, 24) (44, 24)
Training Duration: 0:00:14.247079
{'estimator__subsample': 0.9, 'estimator__n_estimators': 80, 'estimator__min_samples_split': 100, 'estimator__min_samples_leaf': 20, 'estimator__max_features': 4, 'estimator__max_depth': 11, 'estimator__learning_rate': 0.025}
-10.963203343495556


MAE:  11.608
MSE:  333.9813
RMSE:  18.2752
MAPE:  20.7089
Coefficient of determination (R-squared):  0.4384


Round 21: 


(394, 289) (44, 289)
(394, 24) (44, 24)
Training Duration: 0:00:14.744331
{'estimator__subsample': 0.6, 'estimator__n_estimators': 60, 'estimator__min_samples_split': 100, 'estimator__min_samples_leaf': 50, 'estimator__max_features': 4, 'estimator__max_depth': 5, 'estimator__learning_rate': 0.1}
-11.196445531752392


MAE:  9.5724
MSE:  209.729
RMSE:  14.482
MAPE:  18.4834
Coefficient of determination (R-squared):  0.3802


Round

In [50]:
to_excel(mae10,mse10,rmse10,mape10,r210)

In [53]:
#XGBM
start_time = datetime.now()
mae11,mse11,rmse11,mape11,r211 = testing_runs_XGBM(30,data_test,'top12_30_rounds')
end_time = datetime.now()
print('Process Duration: {}'.format(end_time - start_time))

Round 1: 


(394, 289) (44, 289)
(394, 24) (44, 24)
Training Duration: 0:05:02.221096
{'estimator__subsample': 0.1, 'estimator__n_estimators': 400, 'estimator__max_depth': 11, 'estimator__learning_rate': 0.0125, 'estimator__colsample_bytree': 0.30000000000000004}
-9.869392810267538


MAE:  10.7729
MSE:  335.7639
RMSE:  18.3239
MAPE:  19.1922
Coefficient of determination (R-squared):  0.4126


Round 2: 


(394, 289) (44, 289)
(394, 24) (44, 24)
Training Duration: 0:07:00.690919
{'estimator__subsample': 0.7000000000000001, 'estimator__n_estimators': 250, 'estimator__max_depth': 5, 'estimator__learning_rate': 0.05, 'estimator__colsample_bytree': 0.2}
-10.56474943635531


MAE:  8.252
MSE:  145.3162
RMSE:  12.0547
MAPE:  15.7422
Coefficient of determination (R-squared):  0.5328


Round 3: 


(394, 289) (44, 289)
(394, 24) (44, 24)
Training Duration: 0:09:49.131234
{'estimator__subsample': 0.6, 'estimator__n_estimators': 300, 'estimator__max_depth': 3, 'estimator__learning_rate': 0.0125, 'es

Training Duration: 0:06:06.651217
{'estimator__subsample': 0.8, 'estimator__n_estimators': 200, 'estimator__max_depth': 7, 'estimator__learning_rate': 0.025, 'estimator__colsample_bytree': 0.6}
-10.0862642268647


MAE:  9.7409
MSE:  199.3512
RMSE:  14.1192
MAPE:  17.9333
Coefficient of determination (R-squared):  0.4853


Round 23: 


(394, 289) (44, 289)
(394, 24) (44, 24)
Training Duration: 0:06:00.132108
{'estimator__subsample': 0.5, 'estimator__n_estimators': 450, 'estimator__max_depth': 11, 'estimator__learning_rate': 0.025, 'estimator__colsample_bytree': 0.4}
-10.279910489012925


MAE:  8.9672
MSE:  226.2772
RMSE:  15.0425
MAPE:  16.7888
Coefficient of determination (R-squared):  0.5572


Round 24: 


(394, 289) (44, 289)
(394, 24) (44, 24)
Training Duration: 0:05:34.966214
{'estimator__subsample': 0.7000000000000001, 'estimator__n_estimators': 350, 'estimator__max_depth': 5, 'estimator__learning_rate': 0.05, 'estimator__colsample_bytree': 0.4}
-10.28022868614263


MAE:  10.2517


In [54]:
to_excel(mae11,mse11,rmse11,mape11,r211)

In [51]:
#LGBM
start_time = datetime.now()
mae12,mse12,rmse12,mape12,r212 = testing_runs_LGBM(30,data_test,'top12_30_rounds')
end_time = datetime.now()
print('Process Duration: {}'.format(end_time - start_time))

Round 1: 


(394, 289) (44, 289)
(394, 24) (44, 24)
[LightGBM] [Warning] min_child_samples is set=20, min_child_samples= will be ignored. Current value: min_child_samples=20
[LightGBM] [Warning] Unknown parameter: 30
Training Duration: 0:02:34.596170
{'estimator__subsample': 0.6, 'estimator__num_leaves': 4915, 'estimator__n_estimators': 100, 'estimator__min_child_samples ': 30, 'estimator__max_depth': 9, 'estimator__learning_rate': 0.025, 'estimator__colsample_bytree': 0.2}
-10.002442026734608


MAE:  10.6902
MSE:  343.8332
RMSE:  18.5427
MAPE:  19.0449
Coefficient of determination (R-squared):  0.4428


Round 2: 


(394, 289) (44, 289)
(394, 24) (44, 24)
[LightGBM] [Warning] min_child_samples is set=20, min_child_samples= will be ignored. Current value: min_child_samples=20
[LightGBM] [Warning] Unknown parameter: 70
Training Duration: 0:03:46.886886
{'estimator__subsample': 0.1, 'estimator__num_leaves': 307, 'estimator__n_estimators': 100, 'estimator__min_child_samples ': 70, 'estimat

Training Duration: 0:05:18.007642
{'estimator__subsample': 1.0, 'estimator__num_leaves': 19, 'estimator__n_estimators': 200, 'estimator__min_child_samples ': 70, 'estimator__max_depth': 3, 'estimator__learning_rate': 0.0125, 'estimator__colsample_bytree': 0.6}
-9.8470850980187


MAE:  12.2141
MSE:  359.6107
RMSE:  18.9634
MAPE:  22.9267
Coefficient of determination (R-squared):  0.4024


Round 15: 


(394, 289) (44, 289)
(394, 24) (44, 24)
[LightGBM] [Warning] min_child_samples is set=20, min_child_samples= will be ignored. Current value: min_child_samples=20
[LightGBM] [Warning] Unknown parameter: 60
Training Duration: 0:04:01.897353
{'estimator__subsample': 1.0, 'estimator__num_leaves': 5, 'estimator__n_estimators': 300, 'estimator__min_child_samples ': 60, 'estimator__max_depth': 13, 'estimator__learning_rate': 0.025, 'estimator__colsample_bytree': 0.2}
-9.988515171513884


MAE:  11.4403
MSE:  310.9506
RMSE:  17.6338
MAPE:  21.169
Coefficient of determination (R-squared):  0.3382




MAE:  9.4148
MSE:  218.7336
RMSE:  14.7896
MAPE:  17.2002
Coefficient of determination (R-squared):  0.5363


Round 28: 


(394, 289) (44, 289)
(394, 24) (44, 24)
[LightGBM] [Warning] min_child_samples is set=20, min_child_samples= will be ignored. Current value: min_child_samples=20
[LightGBM] [Warning] Unknown parameter: 10
Training Duration: 0:02:47.920048
{'estimator__subsample': 0.4, 'estimator__num_leaves': 1229, 'estimator__n_estimators': 400, 'estimator__min_child_samples ': 10, 'estimator__max_depth': 11, 'estimator__learning_rate': 0.0125, 'estimator__colsample_bytree': 0.30000000000000004}
-10.234337878725302


MAE:  10.8174
MSE:  285.5818
RMSE:  16.8992
MAPE:  20.0393
Coefficient of determination (R-squared):  0.2141


Round 29: 


(394, 289) (44, 289)
(394, 24) (44, 24)
[LightGBM] [Warning] min_child_samples is set=20, min_child_samples= will be ignored. Current value: min_child_samples=20
[LightGBM] [Warning] Unknown parameter: 60
Training Duration: 0:03:05.991079
{'estim

In [52]:
to_excel(mae12,mse12,rmse12,mape12,r212)

## Select k-best

In [25]:
sk = data_1819.copy()
sk = feature_pipeline(sk,2)
sk.head()

,applicable_date,elec_prices,expanding_window_price,wdsp_avg_sqrt,wdsp_avg_sqr,wdsp_g,wdsp_c,wdsp_avg,ng_log,daily_natural_gas,ng_sqr,wdsp_avg_log,ng_sqrt,wdsp_d,Hour
0,2018-09-30 22:00:00,71.267,71.267000,1.756132,9.511056,1.028,3.084,3.084000,1.024664,2.78616,7.762688,1.126227,1.669179,5.140,22
1,2018-09-30 23:00:00,67.212,69.239500,1.851126,11.742044,1.542,3.598,3.426667,1.024664,2.78616,7.762688,1.231588,1.669179,5.140,23
2,2018-10-01 00:00:00,60.500,66.326333,1.896839,12.945604,1.542,4.112,3.598000,1.027864,2.79509,7.812528,1.280378,1.671852,5.140,0
3,2018-10-01 01:00:00,63.682,65.665250,1.896839,12.945604,2.056,3.084,3.598000,1.027864,2.79509,7.812528,1.280378,1.671852,5.654,1
4,2018-10-01 02:00:00,71.617,66.855600,1.655697,7.514908,2.056,1.542,2.741333,1.027864,2.79509,7.812528,1.008444,1.671852,4.626,2


In [26]:
sk.columns

Index(['applicable_date', 'elec_prices', 'expanding_window_price',
       'wdsp_avg_sqrt', 'wdsp_avg_sqr', 'wdsp_g', 'wdsp_c', 'wdsp_avg',
       'ng_log', 'daily_natural_gas', 'ng_sqr', 'wdsp_avg_log', 'ng_sqrt',
       'wdsp_d', 'Hour'],
      dtype='object')

In [27]:
X = sk.iloc[:,2:14]
y = sk['elec_prices']
print(X.shape,y.shape)

(10514, 12) (10514,)


In [28]:
from sklearn.feature_selection import SelectKBest,f_regression
s= SelectKBest(score_func=f_regression,k=5)
X_new = s.fit(X, y)
X.columns[X_new.get_support()]

Index(['expanding_window_price', 'ng_log', 'daily_natural_gas', 'ng_sqr',
       'ng_sqrt'],
      dtype='object')